# Late Fusion Model Embeddings Generation

Purpose of this notebook: Generate embeddings for Late Fusion Model

Based on paper: Soenksen, L.R., Ma, Y., Zeng, C. et al. Integrated multimodal artificial intelligence framework for healthcare applications. npj Digit. Med. 5, 149 (2022). https://doi.org/10.1038/s41746-022-00689-4

Using our best tabular, notes, and imaging models, getting the embeddings and classifications for additional modelling

Goal: model using embeddings alone, or embeddings + classification (approach done in Soenksen et al)

In [1]:
#relevant code chunks to adapt to our purpose

In [155]:
#imports adapted from HAIM API.py
# Base
import cv2
import math
import copy
import pickle
import numpy as np
import pandas as pd
import pandas.io.sql as psql
import datetime as dt
import plotly.express as px
import matplotlib.pyplot as plt
from tqdm import tqdm
from glob import glob
from shutil import copyfile

from dask import dataframe as dd
from dask.diagnostics import ProgressBar
ProgressBar().register()

# Core AI/ML
import tensorflow as tf
import torch
import torch.nn.functional as F
import torchvision, torchvision.transforms #causing problems
from torch.utils.data import Dataset, DataLoader
from torchinfo import summary

# Scipy
from scipy.stats import ks_2samp
from scipy.signal import find_peaks

# Scikit-learn
from sklearn.preprocessing import scale
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer

# NLP
from torch import nn
from transformers import AutoTokenizer, AutoModel, logging
logging.set_verbosity_error()
# os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Computer Vision
import cv2
import skimage, skimage.io
import torchxrayvision as xrv
import timm

# Warning handling
import warnings
warnings.filterwarnings("ignore")

# Data Mapping

In [149]:
target_variables_dict = {
    'no_finding': 0,
    'atelectasis': 1,
    'cardiomegaly': 2,
    'lung_opacity': 3,
    'pleural_effusion': 4,
}

In [262]:
#run processing notebook to get data files
%run processing_data.ipynb

Loading data files... C:/Users/Carolyn/Documents/MIDS/210 Capstone/fusion_data\test_set__chexpert__4_findings__single_label__unbalanced.json test
Loading data files... C:/Users/Carolyn/Documents/MIDS/210 Capstone/fusion_data\train_set__chexpert__4_findings__single_label__balanced.json train
Loading data files... C:/Users/Carolyn/Documents/MIDS/210 Capstone/fusion_data\validation_set__chexpert__4_findings__single_label__unbalanced.json validate
Total Cols
 Index(['patient_id', 'visit_id', 'study_id', 'temperature', 'heartrate',
       'resprate', 'o2sat', 'sbp', 'dbp', 'pain', 'acuity',
       'positive_label_total', 'finding_names', 'radiology_note',
       'discharge_note', 'chief_complaint',
       'major_surgical_or_invasive_procedure', 'history_of_present_illness',
       'past_medical_history', 'family_history', 'atelectasis', 'cardiomegaly',
       'lung_opacity', 'pleural_effusion', 'dataset_type'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16877 en

# Notes

## Notes Model Setup

In [ ]:
#original biobert models
biobert_tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
biobert_model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

In [151]:
# Select the Bio_Discharge_Summary_BERT model
MODEL_CHECKPOINT = 'emilyalsentzer/Bio_ClinicalBERT'

In [152]:
# Select parameters
NUM_CLASSES = 5
MAX_SEQUENCE_LENGTH = 512
NUM_EPOCHS = 15
BATCH_SIZE = 16
LEARNING_RATE = 0.00005

In [192]:
class MulticlassClassification(nn.Module):

    def __init__(self, checkpoint, num_classes, hidden_size=201, dropout_prob=0.3, freeze_bert=True):
        super(MulticlassClassification, self).__init__()

        self.model = AutoModel.from_pretrained(checkpoint)
        self.hidden_size = hidden_size
        self.dropout_prob = dropout_prob
        self.num_classes = num_classes
        self.freeze_bert = freeze_bert

        for param in self.model.parameters():
            param.requires_grad = not self.freeze_bert

        self.pooler_layer = nn.Linear(self.model.config.hidden_size, hidden_size) # maps the output of the BERT model's hidden state to the hidden_size
        self.relu = nn.ReLU() # introduces non-linearity to the model
        self.dropout = nn.Dropout(dropout_prob) # applied for regularization
        self.classification_layer = nn.Linear(hidden_size, num_classes) # projects the hidden_size down to the number of target classes

    def forward(self, input_ids, token_type_ids = None, attention_mask = None):
        outputs = self.model(
            input_ids=input_ids,
            token_type_ids=token_type_ids,
            attention_mask=attention_mask)

        pooler_output = outputs.pooler_output
        hidden = self.pooler_layer(pooler_output)
        hidden = self.relu(hidden)
        hidden = self.dropout(hidden)
        classification = self.classification_layer(hidden) # logits for each class

        return classification, hidden ###STEVEN - ONLY CHANGE I MADE TO THE MODEL

    def unfreeze_bert_layers(self, n_layers):
        """Unfreezes the top n layers of the BERT model."""
        layers_to_unfreeze = list(self.model.encoder.layer[-n_layers:])
        for layer in layers_to_unfreeze:
            for param in layer.parameters():
                param.requires_grad = True

In [182]:
def load_model(model, checkpoint_path):
    model.load_state_dict(torch.load(checkpoint_path, map_location=torch.device('cpu')))
    return model

In [183]:
CHECKPOINT_FOLDER = "./checkpoints"
MODEL_NAME_FOLDER = "./model_findings"

esteban_model = MulticlassClassification(
    checkpoint=MODEL_CHECKPOINT,
    num_classes=NUM_CLASSES,
    freeze_bert=False,
    )

CHECKPOINT_FILE = r"C:\Users\Carolyn\Documents\MIDS\210 Capstone\mids-210-radiology-triage-models-spring24\pretrained_weights\bio_clinical_bert\bio_clinical_bert__balanced__unfrozen_layers__best.pt"

esteban_model.load_state_dict(torch.load(CHECKPOINT_FILE, map_location=torch.device('cpu')))

<All keys matched successfully>

In [184]:
esteban_model.eval()

MulticlassClassification(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, ele

## Notes Embeddings

In [234]:
def get_biobert_classifications(text, model = "MMMM"):
    # Inputs:
    #   text -> Input text (str)
    #
    # Outputs:
    #   classifications -> Final Biobert classifications = (1,num_classifcations)
    #   hidden_embeddings -> Last hidden layer in Biobert model with vector dimensionality = (1,hidder_layer_size)
  
    # %% EXAMPLE OF USE
    # classifications, hidden_embeddings = get_biobert_embeddings(text)
    if model == "HAIM": #to test the HAIM model
        model = biobert_model
    
        tokens_pt = biobert_tokenizer(text, return_tensors="pt")
        outputs = model(**tokens_pt)
        last_hidden_state = outputs.last_hidden_state
        pooler_output = outputs.pooler_output
        classifications = pooler_output.detach().numpy()
        
    else:
        model = esteban_model

        tokens_pt = biobert_tokenizer(text, return_tensors='pt',  add_special_tokens=True, padding='max_length', max_length=MAX_SEQUENCE_LENGTH, truncation=True) # (input_ids, attention_mask, token_type_ids)
        outputs, hidden_outputs = model(**tokens_pt)
        classifications = outputs.detach().numpy()

    return classifications

In [235]:
def get_biobert_embeddings(text, model = "MMMM"):
    # Inputs:
    #   text -> Input text (str)
    #
    # Outputs:
    #   classifications -> Final Biobert classifications = (1,num_classifcations)
    #   hidden_embeddings -> Last hidden layer in Biobert model with vector dimensionality = (1,hidder_layer_size)
  
    # %% EXAMPLE OF USE
    # classifications, hidden_embeddings = get_biobert_embeddings(text)
    if model == "HAIM": #to test the HAIM model
        model = biobert_model
    
        tokens_pt = biobert_tokenizer(text, return_tensors="pt")
        outputs = model(**tokens_pt)
        last_hidden_state = outputs.last_hidden_state
        pooler_output = outputs.pooler_output
        hidden_embeddings = last_hidden_state.detach().numpy()
        
    else:
        model = esteban_model

        tokens_pt = biobert_tokenizer(text, return_tensors='pt',  add_special_tokens=True, padding='max_length', max_length=MAX_SEQUENCE_LENGTH, truncation=True) # (input_ids, attention_mask, token_type_ids)
        outputs, hidden_outputs = model(**tokens_pt)
        hidden_embeddings = hidden_outputs.detach().numpy()

    return hidden_embeddings

In [259]:
#run on EC2 instance
#notes_train_df['notes_classifications'] = notes_train_df.history_of_present_illness.apply(get_biobert_classifications)
#notes_train_df['notes_embeddings'] = notes_train_df.history_of_present_illness.apply(get_biobert_embeddings)
#notes_val_df['notes_classifications'] = notes_val_df.history_of_present_illness.apply(get_biobert_classifications)
#notes_val_df['notes_embeddings'] = notes_val_df.history_of_present_illness.apply(get_biobert_embeddings)
#notes_test_df['notes_classifications'] = notes_test_df.history_of_present_illness.apply(get_biobert_classifications)
#notes_test_df['notes_embeddings'] = notes_test_df.history_of_present_illness.apply(get_biobert_embeddings)

In [260]:
print(notes_train_df.shape)
print(notes_val_df.shape)
print(notes_test_df.shape)

(2086, 6)
(1924, 6)
(1920, 6)


# Image Embeddings

In [263]:
#create Cinthya's model
cinthya_model = torch.load(r"C:\Users\Carolyn\Documents\MIDS\210 Capstone\mids-210-radiology-triage-models-spring24\pretrained_weights\EfficientNet_B3.pth", map_location=torch.device('cpu'))

In [273]:
cinthya_model

EfficientNet(
  (conv_stem): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
        (bn1): BatchNormAct2d(
          40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(40, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
    

In [274]:
#images embeddings
def get_single_chest_xray_embeddings(img, model="HAIM"):
    # Inputs:
    #   img -> Image array
    #
    # Outputs:
    #   densefeature_embeddings ->  CXR dense feature embeddings for image
    #   prediction_embeddings ->  CXR embeddings of predictions for image
    
    # %% EXAMPLE OF USE
    # densefeature_embeddings, prediction_embeddings = get_single_chest_xray_embeddings(img)
    
    # Select if you want to use CUDA support for GPU (optional as it is usually pretty fast even in CPUT)
    cuda = False
    
    # Select model with a String that determines the model to use for Chest Xrays according to https://github.com/mlmed/torchxrayvision
    if model == "MMMM":
        model_state_dict = cinthya_model
    else:
        model_weights_name = "densenet121-res224-chex" # CheXpert (Stanford)
    #model_weights_name = "resnet50-res512-all" # Resnet only for 512x512 inputs
    # NOTE: The all model has every output trained. However, for the other weights some targets are not trained and will predict randomly becuase they do not exist in the training dataset.
    
    # Extract chest x-ray image embeddings and preddictions
    densefeature_embeddings = []
    prediction_embeddings = []
    
    #img = cv2.imread(local_base_dir + r'/' + img)
    img = xrv.datasets.normalize(img, 255)

    # For each image check if they are 2D arrays
    if len(img.shape) > 2:
        img = img[:, :, 0]
    if len(img.shape) < 2:
        print("Error: Dimension lower than 2 for image!")
    
    # Add color channel for prediction
    #Resize using OpenCV
    img = cv2.resize(img, (224, 224), interpolation = cv2.INTER_AREA)   
    img = img[None, :, :]

    #Or resize using core resizer (thows error sometime)
    #transform = transforms.Compose([xrv.datasets.XRayCenterCrop(),xrv.datasets.XRayResizer(224)])
    #img = transform(img)
    if model == "MMMM":
        model = Efficient
    else:
        model = xrv.models.DenseNet(weights = model_weights_name)    # model = xrv.models.ResNet(weights="resnet50-res512-all") # ResNet is also available

    output = {}
    with torch.no_grad():
        img = torch.from_numpy(img).unsqueeze(0)
        if cuda:
            img = img.cuda()
            model = model.cuda()
          
        # Extract dense features
        feats = model.features(img)
        feats = F.relu(feats, inplace=True)
        feats = F.adaptive_avg_pool2d(feats, (1, 1))
        densefeatures = feats.cpu().detach().numpy().reshape(-1)
        densefeature_embeddings = densefeatures

        # Extract predicted probabilities of considered 18 classes:
        # Get by calling "xrv.datasets.default_pathologies" or "dict(zip(xrv.datasets.default_pathologies,preds[0].detach().numpy()))"
        # ['Atelectasis','Consolidation','Infiltration','Pneumothorax','Edema','Emphysema',Fibrosis',
        #  'Effusion','Pneumonia','Pleural_Thickening','Cardiomegaly','Nodule',Mass','Hernia',
        #  'Lung Lesion','Fracture','Lung Opacity','Enlarged Cardiomediastinum']
        preds = model(img).cpu()
        predictions = preds[0].detach().numpy()
        prediction_embeddings = predictions  

    # Return embeddings
    return densefeature_embeddings, prediction_embeddings

In [5]:
#need to for each individual model - save the model to run on images
#or save the final output layers (better, more consistent)

In [265]:
local_base_dir = r"C:\Users\Carolyn\Documents\MIDS\210 Capstone\fusion_data\train_4_bal_s"

In [266]:
img_train_df.dicom_id[0]

'7d6ea06e-554c2ccb-4d9ecefe-eb5ca0e0-7049fa19.jpg'

In [267]:
# Load a sample image
img = cv2.imread(local_base_dir + r'/' + img_train_df.dicom_id[0])
print(img.shape)

(256, 256, 3)


In [272]:
densefeautre_embeddings, prediction_embeddings = get_single_chest_xray_embeddings(img, model = "MMMM")

AttributeError: 'EfficientNet' object has no attribute 'features'

In [ ]:
densefeautre_embeddings

In [22]:
densefeature_embeddings_MFA, prediction_embeddings_MFA = get_single_chest_xray_embeddings(img, model = "MMMM")

AttributeError: 'EfficientNet' object has no attribute 'seek'. You can only torch.load from a file that is seekable. Please pre-load the data into a buffer like io.BytesIO and try to load from it instead.

In [19]:
img_train_df_sample = img_train_df.loc[0:5,]
img_train_df_sample.head()

,patient_id,dicom_id,finding_names
0,11388716,7d6ea06e-554c2ccb-4d9ecefe-eb5ca0e0-7049fa19.jpg,atelectasis
1,11539363,b4b4f64f-ba5163c8-1b7ce58e-a0030af6-4d09dec1.jpg,atelectasis
2,10833304,3b09e882-c5f0d93b-106c7c94-29b839e8-00e7a950.jpg,atelectasis
3,19849119,3100385d-f9e0610a-e32f54df-3c17088b-4da3d491.jpg,atelectasis
4,11749991,c83dc36c-6c58d087-0ef18130-dd3e8cbf-faa5e609.jpg,atelectasis


In [69]:
len(img_train_df)

2086

In [68]:
for i in len(img_train_df_sample)-1:
    
    img_train_df['densefeature_embeddings'], img_train_df_sample['prediction_embeddings'] = get_single_chest_xray_embeddings(patient)
img_train_df_sample.head()

ValueError: too many values to unpack (expected 2)

In [ ]:
def get_chest_xray_embeddings(dt_patient, model="HAIM", verbose=0):
    # Inputs:
    #   dt_patient -> Timebound ICU patient stay structure filtered by max_time_stamp or min_time_stamp if any
    #   verbose -> Level of printed output of function
    #
    # Outputs:
    #   aggregated_densefeature_embeddings -> CXR aggregated dense feature embeddings for all images in timebound patient
    #   densefeature_embeddings ->  List of CXR dense feature embeddings for all images
    #   aggregated_prediction_embeddings -> CXR aggregated embeddings of predictions for all images in timebound patient
    #   prediction_embeddings ->  List of CXR embeddings of predictions for all images
    #   imgs_weights ->  Array of weights for embedding aggregation


    # %% EXAMPLE OF USE
    # aggregated_densefeature_embeddings, densefeature_embeddings, aggregated_prediction_embeddings, prediction_embeddings, imgs_weights = get_chest_xray_embeddings(dt_patient, verbose=2)

    # Clean out process bar before starting
    sys.stdout.flush()

    # Select if you want to use CUDA support for GPU (optional as it is usually pretty fast even in CPUT)
    cuda = False

    # Select model with a String that determines the model to use for Chest Xrays according to https://github.com/mlmed/torchxrayvision
    #   model_weights_name = "densenet121-res224-all" # Every output trained for all models
    #   model_weights_name = "densenet121-res224-rsna" # RSNA Pneumonia Challenge
    #model_weights_name = "densenet121-res224-nih" # NIH chest X-ray8
    #model_weights_name = "densenet121-res224-pc") # PadChest (University of Alicante)
    if model == "HAIM":
        model_weights_name = "densenet121-res224-chex" # CheXpert (Stanford)
    elif model == "EffNet":
        pass
    else:
        model_weights_name = "densenet121-res224-chex" # replace with our model when we can
        
    #   model_weights_name = "densenet121-res224-mimic_nb" # MIMIC-CXR (MIT)
    #model_weights_name = "densenet121-res224-mimic_ch") # MIMIC-CXR (MIT)
    #model_weights_name = "resnet50-res512-all" # Resnet only for 512x512 inputs
    # NOTE: The all model has every output trained. However, for the other weights some targets are not trained and will predict randomly becuase they do not exist in the training dataset.


    # Extract chest x-ray images from timebound patient and iterate through them
    imgs = dt_patient.dicom_id
    densefeature_embeddings = []
    prediction_embeddings = []

    # Iterate
    nImgs = len(imgs)
    with tqdm(total = nImgs) as pbar:
        for idx, img in enumerate(imgs):
            img = skimage.io.imread(img_path) # If importing from path use this
            img = xrv.datasets.normalize(img, 255)
          
            # For each image check if they are 2D arrays
            if len(img.shape) > 2:
                img = img[:, :, 0]
            if len(img.shape) < 2:
                print("Error: Dimension lower than 2 for image!")

            # Add color channel for prediction
            #Resize using OpenCV
            img = cv2.resize(img, (224, 224), interpolation = cv2.INTER_AREA)   
            img = img[None, :, :]
            
            #Or resize using core resizer (thows error sometime)
            #transform = transforms.Compose([xrv.datasets.XRayCenterCrop(),xrv.datasets.XRayResizer(224)])
            #img = transform(img)
            if model == "HAIM":
                model = xrv.models.DenseNet(weights = model_weights_name)
            elif model == "MMMM":
                model = timm.create_model('efficientnet_b3', pretrained=True, num_classes=5)
            else:
                model = xrv.models.DenseNet(weights = model_weights_name)
            # model = xrv.models.ResNet(weights="resnet50-res512-all") # ResNet is also available
            
            output = {}
            with torch.no_grad():
                img = torch.from_numpy(img).unsqueeze(0)
                if cuda:
                    img = img.cuda()
                    model = model.cuda()
              
                # Extract dense features
                feats = model.features(img)
                feats = F.relu(feats, inplace=True)
                feats = F.adaptive_avg_pool2d(feats, (1, 1))
                densefeatures = feats.cpu().detach().numpy().reshape(-1)
                densefeature_embeddings.append(densefeatures) # append to list of dense features for all images
                
                # Extract predicted probabilities of considered 18 classes:
                # Get by calling "xrv.datasets.default_pathologies" or "dict(zip(xrv.datasets.default_pathologies,preds[0].detach().numpy()))"
                # ['Atelectasis','Consolidation','Infiltration','Pneumothorax','Edema','Emphysema',Fibrosis',
                #  'Effusion','Pneumonia','Pleural_Thickening','Cardiomegaly','Nodule',Mass','Hernia',
                #  'Lung Lesion','Fracture','Lung Opacity','Enlarged Cardiomediastinum']
                preds = model(img).cpu()
                predictions = preds[0].detach().numpy()
                prediction_embeddings.append(predictions) # append to list of predictions for all images
            
                if verbose >=1:
                    # Update process bar
                    pbar.update(1)
        
        
    # Get image weights by hours passed from current time to image
    orig_imgs_weights = np.asarray(dt_patient.cxr.deltacharttime.values)
    adj_imgs_weights = orig_imgs_weights - orig_imgs_weights.min()
    imgs_weights = (adj_imgs_weights) / (adj_imgs_weights).max()
  
    # Aggregate with weighted average of ebedding vector across temporal dimension
    try:
        aggregated_densefeature_embeddings = np.average(densefeature_embeddings, axis=0, weights=imgs_weights)
        if np.isnan(np.sum(aggregated_densefeature_embeddings)):
            aggregated_densefeature_embeddings = np.zeros_like(densefeature_embeddings[0])
    except:
        aggregated_densefeature_embeddings = np.zeros_like(densefeature_embeddings[0])
      
    try:
        aggregated_prediction_embeddings = np.average(prediction_embeddings, axis=0, weights=imgs_weights)
        if np.isnan(np.sum(aggregated_prediction_embeddings)):
            aggregated_prediction_embeddings = np.zeros_like(prediction_embeddings[0])
    except:
        aggregated_prediction_embeddings = np.zeros_like(prediction_embeddings[0])
      
      
    if verbose >=2:
        x = orig_imgs_weights
        y = prediction_embeddings
        plt.xlabel("Time [hrs]")
        plt.ylabel("Disease probability [0-1]")
        plt.title("A test graph")
        for i in range(len(y[0])):
            plt.plot(x,[pt[i] for pt in y],'o', label = xrv.datasets.default_pathologies[i])
        plt.legend(bbox_to_anchor=(1.05, 1))
        plt.show()

    # Return embeddings
    return aggregated_densefeature_embeddings, densefeature_embeddings, aggregated_prediction_embeddings, prediction_embeddings, imgs_weights

# Generate Tabular Embeddings

In [5]:
#tabular embeddings
#need to build our own models for these
#for now, append tabular data similarly to demographic data (as raw values)

def get_tabular_embeddings(dt_patient, verbose=0):
    # Inputs:
    #   dt_patient -> Timebound mimic patient structure
    #   verbose -> Flag to print found keyword outputs (0,1)
    #
    # Outputs:
    #   base_embeddings -> Core base embeddings for the selected patient

    # %% EXAMPLE OF USE
    # base_embeddings = get_demographic_embeddings(dt_patient, verbose=1)

    # Retrieve dt_patient and get embeddings 
    demo_embeddings =  dt_patient[['temperature', 'heartrate','resprate', 'o2sat', 'sbp', 'dbp', 'pain', 'acuity']].values[0]
    if verbose >= 1:
        print(demo_embeddings)
    return demo_embeddings


In [9]:
tabular_train_df.head()

,patient_id,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity,finding_names
0,11388716,98.8,106.0,22.0,96.0,93.0,67.0,0,2.0,atelectasis
1,11539363,99.1,80.0,16.0,97.0,162.0,67.0,0,3.0,atelectasis
2,10833304,97.0,98.0,14.0,100.0,159.0,88.0,2,2.0,atelectasis
3,19849119,98.6,92.0,20.0,98.0,127.0,70.0,0,2.0,atelectasis
4,11749991,100.6,110.0,16.0,97.0,166.0,100.0,8,2.0,atelectasis


# Concatenate Embeddings together and save output file

In [6]:
#HAIM's concatentation
def process_cxr_embeddings_patient_id(patient_id, dt_patient, df_init):
    
    # TABULAR EMBEDDINGS EXTRACTION
    tabular_embeddings = get_tabular_embeddings(dt_patient, verbose=0)
    gc.collect() #Clear memory
    
    # NOTES EMBEDDINGS
    aggregated_notes_embeddings, aggregated_notes_hidden_embeddings = get_biobert_embeddings(patient, note_type = 'radnotes')
    gc.collect() #Clear memory

    # CHEST XRAY VISION EMBEDDINGS EXTRACTION
    #aggregated_densefeature_embeddings, _, aggregated_prediction_embeddings, _, _ = get_chest_xray_embeddings(dt_patient, verbose=0)
    #gc.collect() #Clear memory
    
    # CHEST XRAY VISION SINGLE-IMAGE EMBEDDINGS EXTRACTION
    print('getting xray')
    img = df_imcxr[idx]
    densefeature_embeddings, prediction_embeddings = get_single_chest_xray_embeddings(img)
    gc.collect() #Clear memory

    # Create Dataframes filteed by ordered sample number for Fusion
    df_patient_ids_fusion = pd.DataFrame([patient_id],columns=['patient_id'])
    df_tabular_embeddings_fusion = pd.DataFrame(tabular_embeddings.reshape(1,-1), columns=['de_'+str(i) for i in range(demo_embeddings.shape[0])])
    
    df_vision_dense_embeddings_fusion = pd.DataFrame(densefeature_embeddings.reshape(1,-1), columns=['vd_'+str(i) for i in range(densefeature_embeddings.shape[0])])
    df_vision_predictions_embeddings_fusion = pd.DataFrame(prediction_embeddings.reshape(1,-1), columns=['vp_'+str(i) for i in range(prediction_embeddings.shape[0])])
    df_notes_embeddings_fusion = pd.DataFrame(aggregated_notes_embeddings.reshape(1,-1), columns=['n_rad_'+str(i) for i in range(aggregated_rad_embeddings.shape[0])])
    df_notes_hidden_embeddings_fusion = pd.DataFrame(aggregated_notes_hidden_embeddings.reshape(1,-1), columns=['n_rad_'+str(i) for i in range(aggregated_rad_embeddings.shape[0])])

    # Vision targets
    cxr_target_columns = ['split','Atelectasis','Cardiomegaly','Consolidation','Edema','Enlarged Cardiomediastinum','Fracture','Lung Lesion','Lung Opacity','No Finding','Pleural Effusion','Pleural Other','Pneumonia','Pneumothorax','Support Devices', 'PerformedProcedureStepDescription','ViewPosition']
    df_vision_targets_fusion = df_stay_cxr.loc[idx:idx][cxr_target_columns].reset_index(drop=True)

    # Embeddings FUSION
    df_fusion = df_patient_ids_fusion
    df_fusion = pd.concat([df_fusion, df_init], axis=1)
    df_fusion = pd.concat([df_fusion, df_tabular_embeddings_fusion], axis=1)
    df_fusion = pd.concat([df_fusion, df_vision_dense_embeddings_fusion], axis=1)
    df_fusion = pd.concat([df_fusion, df_vision_predictions_embeddings_fusion], axis=1)
    df_fusion = pd.concat([df_fusion, df_notes_embeddings_fusion], axis=1)
    
    #Add targets
    df_fusion = pd.concat([df_fusion, df_vision_targets_fusion], axis=1)
    gc.collect() #Clear memory
    
    return df_fusion


In [82]:
# test tabular "embeddings"

In [83]:
test_tabular_embedding = get_tabular_embeddings(train_df, verbose = 1)

[ 97.6 117.   18.   95.  128.   74.   10.    3. ]
